In [ ]:
import copy

# =====================================================================================
# ENVIRONMENT
# =====================================================================================

import snman
from snman.constants import *
from snman import osmnx_customized as oxc

PERIMETER = 'zrh15'
# set SAVE_TO_DEBUG = True for saving the cached network into the _debug folder
# which is automatically used in the QGIS files
SAVE_TO_DEBUG = True
INTERSECTION_TOLERANCE = 10

# set these paths according to your own setup
data_directory = 'C:/Users/lballo/polybox/Research/SNMan/SNMan Shared/data_v2/'
inputs_path = data_directory + 'inputs/'
if SAVE_TO_DEBUG:
    process_path = data_directory + 'process/' + '_debug' + '/'
else:
    process_path = data_directory + 'process/' + PERIMETER + '/'

oxc.settings.useful_tags_way = OSM_TAGS

In [ ]:
# =====================================================================================
# LOAD DATA
# =====================================================================================

print('Load perimeters')
perimeters = snman.io.load_perimeters(inputs_path + 'perimeters/perimeters.shp')

print('Get data from OSM server')
# At this step, simplification means only removing degree=2 edges
G_raw = oxc.graph_from_polygon(
    # set the perimeter here
    perimeters.to_crs(4326).loc[PERIMETER]['geometry'],
    custom_filter=snman.constants.OSM_FILTER,
    simplify=True, simplify_strict=False, retain_all=True, one_edge_per_direction=False
)

G = copy.deepcopy(G_raw)

print('Load manual intersections')
# polygons used to override the automatically detected intersections in some situations
given_intersections_gdf = snman.io.load_intersections(
    inputs_path + 'intersection_polygons/intersection_polygons.shp'
)

print('Prepare graph')
snman.street_graph.prepare_graph(G)

print('Convert CRS of street graph')
snman.street_graph.convert_crs(G, DEFAULT_CRS)

print('Generate lanes')
# interpreting the OSM tags into a collection of lanes on each edge
snman.space_allocation.generate_lanes(G)

print('Load sensors and assign them to edges in the raw street graph')
sensors_df = snman.io.load_sensors(inputs_path + 'sensors/sensors.csv')
snman.enrichment.match_sensors(G, sensors_df)

In [ ]:
# =====================================================================================
# ENRICH
# =====================================================================================

if 1:
    print('Identify hierarchy')
    # split the edges into hierarchy categories, such as main roads, local roads, etc.
    snman.hierarchy.add_hierarchy(G)

In [ ]:
# =====================================================================================
# CONSOLIDATE PARALLEL AND CONSECUTIVE EDGES
# =====================================================================================
if 1:
    print('Simplify edge geometries')
    snman.simplification.simplify_edge_geometries(G, 5)

for i in range(3):

    print('ITERATION', i)

    print('Detect intersections')
    intersections_gdf = snman.simplification.merge_nodes_geometric(
        G, INTERSECTION_TOLERANCE,
        given_intersections_gdf=given_intersections_gdf
    )

    print('Split through edges in intersections')
    snman.simplification.split_through_edges_in_intersections(G, intersections_gdf)

    print('Detect intersections (repeat to ensure that no points are outside of intersections)')
    intersections_gdf = snman.simplification.merge_nodes_geometric(
        G, INTERSECTION_TOLERANCE,
        given_intersections_gdf=given_intersections_gdf
    )

    print('Add layers to nodes')
    snman.simplification.add_layers_to_nodes(G)

    print('Add connections between components in intersections')
    snman.simplification.connect_components_in_intersections(G, intersections_gdf, separate_layers=True)

    print('Consolidate intersections')
    G = snman.simplification.consolidate_intersections(
        G, intersections_gdf,
        reconnect_edges=True
    )

    print('Merge consecutive edges')
    snman.merge_edges.merge_consecutive_edges(G)

    print('Merge parallel edges')
    snman.merge_edges.merge_parallel_edges(G)

    print('Update precalculated attributes')
    snman.street_graph.update_precalculated_attributes(G)

snman.simplification.simplify_edge_geometries(G, 35)
G = snman.graph_utils.keep_only_the_largest_connected_component(G, weak=True)

In [ ]:
# =====================================================================================
# UPDATE PRE-CALCULATED ATTRIBUTES
# =====================================================================================

print('Add lane stats to edges')
# how many lanes, how wide, etc.
snman.space_allocation.generate_lane_stats(G)

print('Update OSM tags')
# to match the simplified and merged edges
snman.space_allocation.update_osm_tags(G)

if 0:
    print('Update street counts per node')
    # TODO: include in a function for updating pre-calculated attributes, adjust for directed graph
    spn = oxc.stats.count_streets_per_node(G, nodes=G.nodes)
    nx.set_node_attributes(G, values=spn, name="street_count")

In [ ]:
# =====================================================================================
# ENRICH
# =====================================================================================

if 1:
    #TODO: use mapmatching for better performance and accuracy
    #TODO: add route direction for one-way sections
    print('Add public transport')
    pt_network = snman.io.import_geofile_to_gdf(inputs_path + "public_transit/ZVV_LINIEN_GEN_L.shp")
    snman.enrichment.match_pt(G, pt_network)

if 1:
    print('Add elevation')
    G = oxc.elevation.add_node_elevations_raster(G, inputs_path + 'ch_dhm_25/2056/ch_dhm_2056.tif', cpus=1)
    G = oxc.elevation.add_edge_grades(G, add_absolute=False)

if 0:
    print('Add traffic counts')
    source = gpd.read_file(inputs_path + 'traffic_volumes/npvm_2017_filtered.gpkg').to_crs(2056)
    source['fid'] = source.index
    # Remove links with zero traffic (otherwise they will distort the averages on the matched links)
    source = source[source['DTV_ALLE'] > 0]
    snman.enrichment.match_linestrings(G, source, [
        {'source_column': 'DTV_ALLE',   'target_column': 'adt_avg',         'agg': 'avg' },
        {'source_column': 'DTV_ALLE',   'target_column': 'adt_max',         'agg': 'max' },
        {'source_column': 'FROMNODENO', 'target_column': 'npvm_fromnodeno', 'agg': 'list'},
        {'source_column': 'TONODENO',   'target_column': 'npvm_tonodeno',   'agg': 'list'}
    ])

In [ ]:
# =====================================================================================
# EXPORT
# =====================================================================================

if 1:
    print('Export street graph graph')
    # each street is one edge, the lanes are saved as an attribute
    snman.io.export_street_graph(G, process_path + 'street_graph_edges.gpkg', process_path + 'street_graph_nodes.gpkg')

if 1:
    print('Export street raw graph graph')
    # each street is one edge, the lanes are saved as an attribute
    snman.io.export_street_graph(G, process_path + 'raw_street_graph_edges.gpkg', process_path + 'raw_street_graph_nodes.gpkg')

if 1:
    print('Export lane geometries')
    # each lane has an own geometry and with as an attribute, for visualization purposes
    snman.io.export_street_graph_with_lanes(G, KEY_LANES_DESCRIPTION, process_path + 'lane_geometries.shp', scaling=2)

if 1:
    print('Save intersection geometries into a file')
    snman.io.export_gdf(intersections_gdf, process_path + 'intersections_polygons.gpkg', columns=['geometry'])

if 1:
    print('Export as OSM')
    snman.io.export_osm_xml(G, process_path + 'osm.osm', EXPORT_OSM_TAGS, uv_tags=True, tag_all_nodes=False)